In [0]:
import numpy as np
import os
from PIL import Image
import string
from pickle import load,dump



from keras.applications.xception import Xception,preprocess_input
from keras.preprocessing.image import load_img,img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model,load_model
from keras.layers import Dense,Flatten,Input,LSTM,Dropout,Embedding
from keras.layers.merge import add
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Code to load text file into memory

In [0]:
def load_doc(filename):
  file=open(filename,'r')
  text=file.read()
  file.close()
  return text

In [0]:
def all_image_caption(filename):
  file=load_doc(filename)
  captions=file.split('\n')
  descriptions={}
  for caption in captions[:-1]:
    img,caption=caption.split('\t')
    if img[:-2] not in descriptions:
      descriptions[img[:-2]]
    else:
      decriptions.append[caption]
  return descriptions

In [0]:
def cleaning_text(captions):
  table=str.maketrans(",",string.puntuation)
  for img,caps in captions.items():
    for i,img_caption in enumerate(caps):
      img_caption.replace("-"," ")
      desc = img_caption.split()
      #converts to lowercase
      desc = [word.lower() for word in desc]
      #remove punctuation from each token
      desc = [word.translate(table) for word in desc]
      #remove hanging 's and a 
      desc = [word for word in desc if(len(word)>1)]
      #remove tokens with numbers in them
      desc = [word for word in desc if(word.isalpha())]
      #convert back to string
      img_caption = ' '.join(desc)
      captions[img][i]= img_caption
  return captions

In [0]:
def text_vocabulary(descriptions):
  vocab=set()
  for key in descriptions.keys():
    [vocab.update(d.split()) for d in descriptions[key]]
  
  return vocab

In [0]:
def save_descriptions(descriptions,filename):
  lines=list()
  for key,desc_list in descriptions.items():
    for desc in desc_list:
      lines.append(key + '\t' + desc)
  
  data='\n'.join(lines)
  file=open(filename,'w')
  file.write(data)
  file.close()

In [0]:
dataset_images="/content/drive/My Drive/Colab Notebooks/Flickr8k_text"
dataset_text="/content/drive/My Drive/Colab Notebooks/Flickr8k_text"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
filename=dataset_images+'/'+'Flickr8k.token.txt'
descriptions=all_image_caption(filename)

In [0]:
print('length of descripiton',len(descriptions))
clean_description=cleaning_text(descriptions)


vocabulary=text_vocabulary(clean_description)
save_descriptions(clean_description,"descriptions.txt")

Extracting Features from the dataset_images

In [0]:
def extract_feature(directory):
  model=Xception(include_top=False,pooling='avg')
  features={}
  for img in tqdm(os.listdir(directory)):
    filename=directory+'/'+img
    image=Image.open(filename)
    image=image.resize((299,299))
    image=np.expand_dims(image,axis=0)
    image=image/127.5
    image=image-1.0

    feature=maodel.predict(image)
    features[img]=feature

  return features

In [0]:
features=extract_feature(dataset_images)
dump(features,open("features.p","wb"))

In [0]:
features=load(open("features.p","rb"))

In [0]:
#load the data 
def load_photos(filename):
    file = load_doc(filename)
    photos = file.split("\n")[:-1]
    return photos

def load_clean_descriptions(filename, photos): 
    #loading clean_descriptions
    file = load_doc(filename)
    descriptions = {}
    for line in file.split("\n"):
        words = line.split()
        if len(words)<1 :
            continue
        image, image_caption = words[0], words[1:]
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = '<start> ' + " ".join(image_caption) + ' <end>'
            descriptions[image].append(desc)
    return descriptions

def load_features(photos):
    #loading all features
    all_features = load(open("features.p","rb"))
    #selecting only needed features
    features = {k:all_features[k] for k in photos}
    return features

filename = dataset_text + "/" + "Flickr_8k.trainImages.txt"
#train = loading_data(filename)
train_imgs = load_photos(filename)
train_descriptions = load_clean_descriptions("descriptions.txt", train_imgs)
train_features = load_features(train_imgs)

In [0]:
#loading the data for images

def load_photos(filename):
  file=load_doc(filename)
  photos=file.split("\n")[:-1]
  return photos

In [0]:
def load_clean_description(filename,photos):
  file=load_doc(filename)
  descriptions={}
  for line in file.split("\n"):
    words=lines.split()
    if len(words)<1:
      continue

    
    image,image_caption=words[0],words[1:]

    if image in photos:
      if image not in descriptions:
        descriptions[image]=[]
      desc='<start> '+" ".join(image_caption)+' <end>'
      descriptions[image].append(desc)
    
  return descriptions

In [0]:
def load_features(photos):
  all_features=load(open("features.p"."rb"))
  features = {k:all_features[k] for k in photos}
  return features

In [0]:
filename = dataset_text + "/" + "Flickr_8k.trainImages.txt"

train_imgs=load_photos(filename)
train_descriptions=load_clean_description("descriptions.txt",train_imgs)
train_features=load_features(train_imgs)

Tokenizing The vocabulary

Computers don’t understand English words, for computers, we will have to represent them with numbers. So, we will map each word of the vocabulary with a unique index value. Keras library provides us with the tokenizer function that we will use to create tokens from our vocabulary and save them to a “tokenizer.p” pickle file.

In [0]:
def dict_to_list(descriptions):
  all_desc=[]
  for key in descriptions.keys():
    [all_desc.append(d) for d in descriptions[key]]
  return all_desc

In [0]:
from keras.preprocessing.text import Tokenizer

In [0]:
def create_tokenizer(descriptions):
  desc_list=dict_to_list(descriptions)
  tokenizer=Tokenizer()
  tokenizer.fit_on_texts(desc_list)
  return tokenizer

In [0]:
tokenizer=create_tokenizer(train_descriptions)
dump(tokenizer,open("tokenizer.p","wb"))
vocab_size=len(tokenizer.word_index)+1
vocab_size

In [0]:
def max_length(descriptions):
  desc_list=dict_to_list(descriptions)
  return max(len(d.split()) for d in desc_list)

In [0]:
max_len=max_length(descriptions)
max_len

**Create Data generator**

Let us first see how the input and output of our model will look like. To make this task into a supervised learning task, we have to provide input and output to the model for training. We have to train our model on 6000 images and each image will contain 2048 length feature vector and caption is also represented as numbers. This amount of data for 6000 images is not possible to hold into memory so we will be using a generator method that will yield batches.

The generator will yield the input and output sequence

In [0]:
def data_generator(descriptions,tokennizer,features,max_length):
  while 1:
    for key,description_list in descriptions.items():
      feature=features[key][0]
      for key, description_list in descriptions.items():
            #retrieve photo features
            feature = features[key][0]
            input_image, input_sequence, output_word = create_sequences(tokenizer, max_length, description_list, feature)
            yield [[input_image, input_sequence], output_word]

In [0]:
def create_sequence(tokenizer,max_length,desc_list,feature):
  X1,X2,y=list(),list(),list()
  for desc in desc_list:
    seq=tokenizer.text_to_sequence([desc])[0]
    for i in range(1,len(seq)):
      in_seq,out_seq=seq[i],seq[:i]
      in_seq=pad_sequence([in_seq],maxlen=max_length)[0]
      out_seq=to_categorical([out_seq],num_classes=vocab_size)[0]
      X1.append(feature)
      X2.append(in_seq)
      y.append(out_seq)
  return np.array(X1),np.array(X2),np.array(y)

[a,b],c=next(data_generator(descriptions,tokenizer,features,max_length))

a.shape ,b.shape,c.shape()

In [0]:
from keras.utils import plot_model

def define_model(vocab_size,max_length):
  inputs1=Input(shape=(2048,))
  fe1=Dropout(0.5)(inputs1)
  fe2=Dense(256,activation='relu')(fe1)

  inputs2=Input(shape=(max_length,))
  se1=Embedding(vocab_size,256,mark_zero=True)(inputs2)
  se2=Dropout(0.5)(se1)
  se3=LSTM(256)(se2)


  decoder1=add(fe1,se3)
  decoder2=Dense(256,activation='relu')(decoder1)
  decoder3=Dense(vocab_size,ativtation='softmax')(decoder2)

  model=Model(inputs=(inputs1,inputs2),outputs=decoder3)
  model.compile(loss='categorical_crossentropy',optimizer='adam')


  print(model.summary())
  plot_model(model,to_file='model.png',show_shapes=True)

  return model

To train the model, we will be using the 6000 training images by generating the input and output sequences in batches and fitting them to the model using model.fit_generator() method. We also save the model to our models folder. This will take some time depending on your system capability

In [0]:
# train our model
print('Dataset: ', len(train_imgs))
print('Descriptions: train=', len(train_descriptions))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)



model=define_model(vocab_size,max_length)
epochs=10
steps=len(train_desriptions)
os.mkdir("models")


for i in range(epochs):
  generator=data_generator(train_descriptions,train_festures,tokenizer,max_length)
  model.fit_generator(generator,epochs=1,steps_per_epochs=steps,verbose=1)
  model.save("models/model_" + str(i) + ".h5")